# JPX Tokyo Stock Exchange Prediction <span style="color:DarkCyan"> with LGBM</span>

Thank you for viewing my notebook, I hope you enjoy it 📊<br>
Don't hesitate to leave any feedback 😉

# Table of Contents
1. [Overview](#Overview)
1. [Load JPX data](#Load-JPX-data)
2. [Preprocess](#Preprocess)
3. [Train the LGBM Model](#Train-the-LGBM-Model)
4. [Predict Test Data](#Predict-Test-Data)
5. [Submit](#Submit)

# Overview

In this notebook, I will build a <span style="color:DarkCyan">Light Gradient Boosting Model</span> for the [JPX Tokyo Stock Exchange Prediction Competition](https://www.kaggle.com/competitions/jpx-tokyo-stock-exchange-prediction).

### Quick introduction to <span style="color:DarkCyan">LGBM</span>

[Light GBM](https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc) is a gradient boosting model that uses decision tree algorithm.<br>
<span style="color:DarkCyan">Light GBM</span> grows tree vertically while other gradient boosting algorithms (e.g. XGBoost) grow trees horizontally.<br>
<span style="color:DarkCyan">LGBM</span> chooses the leaf with max delta loss to grow. Holding leaf fixed, leaf-wise algorithms tend to achieve lower loss than level-wise algorithms.

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import jpx_tokyo_market_prediction

pd.set_option('display.max_columns', 100)

In [2]:
# 我亂改版
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # gbdt - traditional Gradient Boosting Decision Tree
    'objective': 'regression',  # L2 loss
    'metric': 'mae',
    'learning_rate': 0.1,
    'lambda_l1': 0.5,  # L1 regularization
    'lambda_l2': 0.5,  # L2 regularization
    'num_leaves': 100,
    'feature_fraction': 0.5,  # LightGBM will select 50% of features before training each tree
    'bagging_fraction': 0.5,  # LightGBM will select 50% part of data without resampling
    'bagging_freq': 5,  #  perform bagging at every k iteration
    'min_child_samples': 10,
    'seed': 42
}
# 原始版
# lgbm_params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',  # gbdt - traditional Gradient Boosting Decision Tree
#     'objective': 'regression',  # L2 loss
#     'metric': 'rmse',
#     'learning_rate': 0.05,
#     'lambda_l1': 0.5,  # L1 regularization
#     'lambda_l2': 0.5,  # L2 regularization
#     'num_leaves': 10,
#     'feature_fraction': 0.5,  # LightGBM will select 50% of features before training each tree
#     'bagging_fraction': 0.5,  # LightGBM will select 50% part of data without resampling
#     'bagging_freq': 5,  #  perform bagging at every k iteration
#     'min_child_samples': 10,
#     'seed': 42
# }


Load datasets

# Load JPX data

In [3]:
file_path = '/kaggle/input/jpx-tokyo-stock-exchange-prediction/'
prices = pd.read_csv(Path(file_path, 'train_files/stock_prices.csv'))
stock_list = pd.read_csv(Path(file_path, 'stock_list.csv'))

In [4]:
prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


Display information about main dataset

In [5]:
prices.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332531 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   RowId             2332531 non-null  object 
 1   Date              2332531 non-null  object 
 2   SecuritiesCode    2332531 non-null  int64  
 3   Open              2324923 non-null  float64
 4   High              2324923 non-null  float64
 5   Low               2324923 non-null  float64
 6   Close             2324923 non-null  float64
 7   Volume            2332531 non-null  int64  
 8   AdjustmentFactor  2332531 non-null  float64
 9   ExpectedDividend  18865 non-null    float64
 10  SupervisionFlag   2332531 non-null  bool   
 11  Target            2332293 non-null  float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 198.0+ MB


# Preprocess

Manage data id: add variable describing how many days have passed from the beginning of the dataset 

In [6]:
# prices['Date']就是一串時間
prices['Date'] = pd.to_datetime(prices['Date'])
min_date = prices['Date'].min()
prices['date_rank'] = (prices['Date'] - min_date).dt.days
# date rank:ex 2017-01-04都是第一名

In [7]:
# mask = prices['Date'] >= "2019-12-31"
# prices = prices[mask]

In [8]:
prices

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,date_rank
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,0
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324,0
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154,0
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053,0
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816,1794
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478,1794
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302,1794
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098,1794


In [9]:
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'date_rank', 'SecuritiesCode']

Choose features to use as independent variables in the model

In [10]:
# dropna拔掉空值
prices = prices.dropna(subset=features)

In [11]:
# pop出target，prices中就沒有target了
target = prices.pop('Target')
target_mean = target.mean()

In [12]:
# train_test_split分割成training data 和testing data
train_f, valid_f = train_test_split(prices[features], test_size=0.2)
train_idx = train_f.index
valid_idx = valid_f.index
# 直接告訴他正確答案，所以這個model預測的是target
lgb_train = lgb.Dataset(train_f, target[train_idx])
lgb_valid = lgb.Dataset(valid_f, target[valid_idx], reference=lgb_train)

In [13]:
train_f.head()

,Open,High,Low,Close,Volume,date_rank,SecuritiesCode
618790,4455.0,4460.0,4420.0,4450.0,5900,488,7564
2216937,1980.0,2080.0,1978.0,2075.0,156600,1709,3458
2051298,1160.0,1186.0,1041.0,1079.0,157800,1589,4800
669843,845.0,851.0,840.0,841.0,94500,525,6330
395741,1411.0,1437.0,1401.0,1422.0,16800,309,1712


# Train the <span style="color:DarkCyan">LGBM</span> Model

In [14]:
# train
model = lgb.train(
    lgbm_params,
    lgb_train,
    valid_sets=[lgb_train, lgb_valid],
    valid_names=['Train', 'Valid'],
    num_boost_round=2000,
    early_stopping_rounds=100,
    verbose_eval=100,
)

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.186515 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1785
[LightGBM] [Info] Number of data points in the train set: 1859938, number of used features: 7
[LightGBM] [Info] Start training from score 0.000421
Training until validation scores don't improve for 100 rounds
[100]	Train's l1: 0.0149924	Valid's l1: 0.015024
[200]	Train's l1: 0.014957	Valid's l1: 0.0150187
Early stopping, best iteration is:
[189]	Train's l1: 0.0149591	Valid's l1: 0.0150183


# Predict Test Data

In [15]:
test_prices = pd.read_csv(Path(file_path, 'example_test_files/stock_prices.csv'))
test_prices['date_rank'] = (pd.to_datetime(test_prices['Date']) - min_date).dt.days

In [16]:
# 預測出大家的target
preds =  model.predict(test_prices[features], num_iteration=model.best_iteration)
preds

array([0.01281833, 0.01047946, 0.01209424, ..., 0.01363076, 0.01125501,
       0.01259772])

In [17]:
# 有幾個人缺值
pd.Series(preds).fillna(target_mean).rank(ascending = False,method = 'first').astype(int)

0        900
1       2570
2       1489
3        319
4        925
        ... 
3995     556
3996     201
3997     409
3998    2061
3999    1068
Length: 4000, dtype: int64

# Submit

In [18]:
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    prices['date_rank'] = (pd.to_datetime(prices['Date']) - min_date).dt.days
    preds = model.predict(prices[features], num_iteration=model.best_iteration)
    preds = np.squeeze(preds)
    print(preds)
    sample_prediction["Prediction"] = preds
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    env.predict(submission)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
[0.01281833 0.01047946 0.01209424 ... 0.0125299  0.01306043 0.01301532]
[0.01315223 0.01205904 0.01176459 ... 0.01363076 0.01125501 0.01259772]
